In [152]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 13 17:14:17 2021

@author: tasfi
"""

import matplotlib
matplotlib.use("Agg")
from matplotlib import colors, ticker, patches
from matplotlib import patheffects, gridspec
import matplotlib.dates as mdates
import numpy as np
import matplotlib.pyplot as plt
import iris # minimal fio done here 
import iris.quickplot as qplt
import cartopy.crs as ccrs
from datetime import datetime, timedelta
import numpy as np
from netCDF4 import Dataset
#from oceans import wrap_lon180
import oceans
import mplleaflet
#import Leaflet as L
import pandas as pd 
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import branca.colormap
from collections import defaultdict
import webbrowser
#import pylab
#import time
#from numba import jit, cuda 
#wind_U = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/10m_uwind.20200102T1500Z.nc")
#wind_V = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/10m_vwind.20200102T1500Z.nc")
#fire = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/firefront.20200102T1500Z.nc")

#ro1_cubes = iris.load("C:/PHD/APR-BOM/Interactive-Map-Visualization-using-Folium-main/sampledata_ro1.nc")
#ro1_cubes = iris.load("C:/PHD/APR-BOM/visualize/my codes/umnsaa_2020010215_mdl_ro1.nc")

In [153]:
import xarray
wind_u = xarray.open_dataset("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/10m_uwind.20200102T1500Z.nc")
wind_v = xarray.open_dataset("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/10m_vwind.20200102T1500Z.nc")
fire = xarray.open_dataset("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/firefront.20200102T1500Z.nc")
print(np.max(wind_u.UWIND_2.data))
print(np.min(wind_u.UWIND_2[0].data))

30.273590243411864
-13.409117820080075


In [154]:
firei = iris.load("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/fire/firefront.20200102T1500Z.nc")
#print(wind_u)
print(np.max(firei[0].data))
print(np.min(firei[0].data))
print('\n')
#print(wind_v)
fireii = firei.extract("firefront")[0] 
fireI = fireii[0]
print(np.min(fireii.data))
print(np.max(fireii.data))

0.00839999970048666
-0.050337099538912265


-0.050337099538912265
0.00839999970048666


In [237]:
import math

#x_wind0 = wind_u.extract("UWIND_2")[0] # extract returns list of cubes
#y_wind0 = wind_v.extract("VWIND_2")[0]

#y_wind1 = ro1_cubes.extract("y_wind")[0]
#x_wind = x_wind0[0,0]   #######x_wind0[time-dim,level-dim]

#create map
map0 = folium.Map(location=[-35.7, 136.4],#tiles="Cartodb Positron",
                  zoom_start=10)

#legend
steps=30
step = branca.colormap.StepColormap(
 colors=['cyan','yellow','BlueViolet','lime','magenta','teal'
                                                   ,'DarkTurquoise','green','indigo',
                                                  'brown'],                                          
 vmin=0, vmax=27,
 index=[0,3,6,9,12,15,18,21,24,27],  #for change in the colors, not used fr linear
 caption='10m-Wind Speed'    #Caption for Color scale or Legend
)
colormap = branca.colormap.LinearColormap(['blue', 'cyan', 'green', 'yellow','orange', 'red'],
                               index=[0, .2,.4, .6, .8,.95])

colormap0 = branca.colormap.LinearColormap(colors=['cyan','yellow','BlueViolet',
                                                   'lime','magenta','orange'
                                                   ,'red','green','indigo',
                                                  'brown'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Wind Speed')#.to_step(steps)
colormap1 = branca.colormap.LinearColormap(colors=['cyan','yellow','BlueViolet',
                                                   'lime','magenta','springGreen'
                                                   ,'teal','green','indigo',
                                                  'brown'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Wind Speed')#.to_step(steps)

colormap9 = branca.colormap.LinearColormap(colors=['Plum','violet','DarkOrchid',
                                                   'fuchsia','MediumOrchid','MediumPurple'
                                                   ,'BlueViolet','DarkMagenta','indigo',
                                                  'MediumSlateBlue'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Wind Speed')#.to_step(steps)
colormap1._repr_html_()

gradient_map=defaultdict(dict)

for i in range(steps):
        gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
    
 #add color bar at the top of the map
colormap0.add_to(map0)
#step.add_to(map0)


#wind direction quivers
def quiver1(x, y, U, V,feature_group0):
    fig, ax = plt.subplots()
       
    kw = dict(color='black', alpha=0.8, scale=1)
    q = ax.quiver(x, y, U, V, **kw)
    #plt.savefig("C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/example_quiver.png")
    gj = mplleaflet.fig_to_geojson(fig=fig)

    count = 0
    count1 = 0
    b = 1
    #for 
    
    for feature in gj['features']: 
        #k=0
        if count1<=60 and b%6 == 0:
            if count%2==0:
                if feature['geometry']['type'] == 'Point':                
                    lon, lat = feature['geometry']['coordinates']
                #print(lat)   
                    #print(lon.shape)
                    div = feature['properties']['html']

                    icon_anchor = (feature['properties']['anchor_x'],
                                feature['properties']['anchor_y'])

                    icon = folium.features.DivIcon(html=div,
                                            icon_anchor=icon_anchor)
                    marker = folium.Marker([lat, lon], icon=icon)
                    feature_group0.add_child(marker)
        
            #else:
                #msg = "Unexpected geometry {}".format
                #raise ValueError(msg(feature['geometry']))
        if count1 == 60:
            count1 = 0
            b += 1
        #print(k)
        count += 1
        count1 += 1
    #fig.clf()
    #plt.ion()
    #plt.cla()
    #plt.clf()
    #plt.close(fig)
    #fig.clf()
    return feature_group0

def map_fire(ff,lats,lons, **contourargs):
    """   
    Plot fire outline onto either an XY grid or geoaxes (eg from map_tiff)
    if mapping latlons onto geoaxes transform needs to be True
    INPUTS:
        ff : 2d array [lats, lons]
        lats,lons : 1darray (degrees)
        
    """
    if ff is None:
        return None
    # unmask if necessary
    if np.ma.is_masked(ff):
        ff = ff.data
    # default fire colour:
    if 'colors' not in contourargs:
        contourargs['colors']='red'
    fflatlon = ff
    fireline = None
    # May need to transpose the fire
    if (ff.shape[0] == len(lons)) and (ff.shape[0] != len(lats)):
        fflatlon = ff.T
        print("WARNING: fire is in [lon,lat]")
    # only plot if there is fire
    if np.sum(ff<0) > 0:
        fireline = plt.contour(lons,lats,fflatlon,np.array([0]),**contourargs)
    
    return fireline 

#@jit(target ="cuda") 
#def compute():

index = 1
k = 479
fireii = firei.extract("firefront")[0] 
for i in range(index):
    #x_wind = x_wind0[0] #subset to first time
    #y_wind = y_wind0[0]
    if i%20==0:       
        x_wind = wind_u.UWIND_2[k]
        y_wind = wind_v.VWIND_2[k]
    #ff = fire.firefront[0]
    #print(x_wind)
    #print(x_wind.data)
    
    
    #x_wind = x_wind#*10000000000000000000
    #y_wind = y_wind#*10000000000000000000
    
    #x_wind = x_wind#*10000000000
    #y_wind = y_wind#*10000000000
    
        lats,lons = x_wind.lat, x_wind.lon
    #print(lats)
        speed = (x_wind**2 + y_wind**2)**0.5
    #print(speed)
    #speed = iris.analysis.maths.apply_ufunc(np.hypot,x_wind,y_wind)
    
        u = x_wind
        v = y_wind
        lon = lons
        lat = lats
        lat, lon = np.meshgrid(lat, lon)
        mask_x = np.logical_and(lon > 136, lon < 138)
    #print(mask_x)
        mask_y = np.logical_and(lat > -38, lat < -35)
    #print(mask_y)
        mask = np.logical_and(mask_x, mask_y)
        print(mask.shape)
    
        x = lon[mask]
        y = lat[mask]
        u.data = u.data/128 #reduced the vectors by power of 2 for reducing browser load
        v.data = v.data/128
        U = u.data[mask]#.filled(fill_value=0)
        V = v.data[mask]#.filled(fill_value=0)
        #print(U.shape)
    #count = 0
    #for k in range(len(speed)):
        #if k%4==0:
            #U[k] = 0.0
            #V[k] = 0.0
            #U[k] = u[k].data[mask]#.filled(fill_value=0)
            #V[k] = v[k].data[mask]#.filled(fill_value=0)
        #count+=1
    
    
        speed.data = speed.data
        S = speed.data[mask]/30#.filled(fill_value=0)
        print(np.max(S.data))
    #print(len(S))
    #print(U)
        
        feature_group0 = folium.FeatureGroup(name='quiver')
        feature_group0 = quiver1(x, y, U, V,feature_group0)      
        map0.add_child(feature_group0)
        plt.clf()       
        
    
        heatmap_wide = HeatMap(list(zip(y.data, x.data, S.data)),
                        min_opacity=0,max_opacity=100,
                        radius=5,#10,#8,
                               #gradient= {'0.0':'Plum','0.1':'violet','0.2':'darkOrchid',
                                                  # '0.3':'fuchsia','0.4':'mediumOrchid','0.5':'mediumPurple '
                                                   #,'0.6':'blueViolet','0.7':'darkMagenta','0.8':'indigo',
                                                  #'0.9':'MediumSlateBlue'},
                           gradient= {'0.0':'cyan','0.1':'yellow','0.2':'blueviolet',
                                                   '0.3':'lime','0.4':'magenta','0.5':'orange',#'teal'
                                                   '0.6':'red'#'DarkTurquoise'
                                      ,'0.7':'green','0.8':'indigo',
                                                  '0.9':'brown'},blur=0, 
                        max_zoom=.5, 
                         )
    
        map0.add_child(heatmap_wide)
        print(i)

        #map0.add_child(folium.map.LayerControl())

        map0.save(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\wind speed n direction\windSpeednDirection_' + str(k) + '.html')
        #map0.save (r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\fire\windSpeednDirection.html')
        #webbrowser.open(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\wind speed n direction\windSpeednDirection' + str(i) + '.html')
#map0

(144, 60)
0.5171207231428044
0


In [199]:

from matplotlib import patheffects, colors, image, ticker

#create map
#map0 = folium.Map(location=[-35.5, 136.4],#tiles="Cartodb Positron",
                  #zoom_start=9)

#legend
steps=33
colormap = branca.colormap.LinearColormap(['indigo','blue','lightblue',
                                           'cyan','green','darkgreen',
                                           'yellow','orange','red','magenta',
                                           'purple','pink','white','black','black','black'],
                               index=[0.0,0.03,0.06,0.09,0.12,0.15,
                                      0.18,0.21,0.24,0.27,0.30,0.33,0.36,.4,.6,.8])
colormap._repr_html_()



colormap1 = branca.colormap.LinearColormap(colors=['cyan','yellow','BlueViolet',
                                                   'lime','magenta','orange'
                                                   ,'red','green','indigo',
                                                  'brown'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Horizontal Wind Speed')#.to_step(steps)

colormap0 = branca.colormap.LinearColormap(colors=['Plum','violet','DarkOrchid',
                                                   'fuchsia','MediumOrchid','MediumPurple'
                                                   ,'BlueViolet','DarkMagenta','indigo',
                                                  'MediumSlateBlue'],
                                          index=[0,3,6,9,12,15,18,21,24,27],vmin=0,vmax=27
                                            ,caption='Horizontal Wind Speed')#.to_step(steps)
        
    
#gradient_dict = {}

step = branca.colormap.StepColormap(
 colors=['indigo','blue','lightblue','cyan','green','lightgreen','yellow','orange','red','magenta','purple','pink','white'],                                          
 vmin=0, vmax=360,
 index=[0,30,60,90,120,150,180,210,240,270,300,330,360],  #for change in the colors, not used fr linear
 caption='Color Scale for Map'    #Caption for Color scale or Legend
)

#colormap.add_to(map0) 
#colormap1.add_to(map0) 
#step.add_to(map0)      
#linear.add_to(map0)
#branca.colormap.linear

end =[]
tmp =[]
tmp2 = [] 
time = 480
#x_wind0 = wind_U.extract("UWIND_2")[0]
#y_wind0 = wind_V.extract("VWIND_2")[0]
fireii = firei.extract("firefront")[0] 
#k = 
for i in range(1):
    if i%1==0:

        fireI = fireii[k]
        #print(fireI.data.shape)

        lats,lons = fireI.coord('latitude').points, fireI.coord('longitude').points
        #u = x_wind
        #v = y_wind
        lon = lons
        lat = lats
        lat, lon = np.meshgrid(lat, lon)
        mask_x = np.logical_and(lon > 136, lon < 138)
    #print(mask_x)
        mask_y = np.logical_and(lat > -38, lat < -35)
    #print(mask_y)
        mask = np.logical_and(mask_x, mask_y)
    #print(mask)
    
        x = lon[mask]
        y = lat[mask]
    
        F = fireI.data[mask]#.filled(fill_value=0)
        
        #if np.any(fireii[i].data < 0):
        #if np.any(F.data < 0):
            #for j in range(8640):
                #if F[j]<0:
                    #F[j] = (6 + F[j])*0.1
                    #print(F[j])
                #else:
                    #F[j] = (1 - F[j])*0.01
                #out = np.array([y[j], x[j], F[j]]).tolist()
                ##print(F[j])
                #end.append(out)           
                #tmp.append(out)
                #end=[]
            
            #tmp2.append(tmp)
            #tmp = []
            
        for j in range(8640):
            if F[j]<0:
                #print('y')
                F[j] = (6 + F[j])*0.1
                #print(F[j])
            else:
                F[j] = (1 - F[j])*0.01
            #out = np.array([y[j], x[j], F[j]]).tolist()
                #print(F[j])
            #end.append(out)           
            #tmp.append(out)
            #end=[]
        #tmp2.append(tmp)
        #tmp = []
                
    #if np.any(fireI.data < 0):
        #out = np.array([lats, lons, fireI.data.T]).tolist()
        #end.append(out)           
        #tmp.append(out)
        #end=[]
        
    
        #plt.contour(lons,lats,ffti,np.array([0]),transform=ccrs.PlateCarree(),colors='red')
    
        
    #print(i)
#print(tmp2)
heatmap_fire = HeatMap(list(zip(y.data, x.data, F)),
                        min_opacity=0.1,max_opacity=50,
                        radius=8,
                               #gradient= {'0.0':'Plum','0.1':'violet','0.2':'darkOrchid',
                                                  # '0.3':'fuchsia','0.4':'mediumOrchid','0.5':'mediumPurple '
                                                   #,'0.6':'blueViolet','0.7':'darkMagenta','0.8':'indigo',
                                                  #'0.9':'MediumSlateBlue'},
                           gradient= {'0.0':'lightblue','.49':'orange','.5991':'darkorange','.5994':'tomato',
                                                     '.5997':'orangered','.5999':'red','0.9':'red'},
                       #gradient= {'0.0':'lightblue','.49':'Darkgray','.5991':'Darkgray','.5994':'Darkgray',
                                                     #'.5997':'Darkgray','.5999':'Darkgray','0.9':'Darkgray'},
                       blur=0, 
                        max_zoom=1, 
                         )
    
map0.add_child(heatmap_fire)
map0.add_child(heatmap_wide)    

#map0.add_child(folium.map.LayerControl())


map0.save(r'C:\PHD\APR-BOM\visualize\my codes\exploratory_KI_run1\figures\windSpeednDirectionNfire\windSpeednDirecnFire_' + str(k) + '.html')
map0     

In [26]:
image_path = 'C:/PHD/APR-BOM/visualize/my codes/exploratory_KI_run1/figures/wind speed n direction/'

images = list(image_path.glob('*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))
    os.remove(file_name)
    
imageio.mimwrite('GifMap.gif', image_list, fps=2)

AttributeError: 'str' object has no attribute 'glob'

In [ ]:
else:
            if feature['geometry']['type'] == 'Point':                
                lon, lat = feature['geometry']['coordinates']
                #print(lat)
                if count1%60 == 0: 
                    lon1 = lon
                    lat1 = lat
                    k += 1
                    
                    #print(lon.shape)

                    div = feature['properties']['html']

                    icon_anchor = (feature['properties']['anchor_x'],
                                feature['properties']['anchor_y'])

                    icon = folium.features.DivIcon(html=div,
                                            icon_anchor=icon_anchor)
                    marker = folium.Marker([lat1, lon1], icon=icon)
                    feature_group0.add_child(marker)
            
        

In [89]:
import pylab
import time
import random
import matplotlib.pyplot as plt

dat=[0,1]
fig = plt.figure()
ax = fig.add_subplot(111)
Ln, = ax.plot(dat)
ax.set_xlim([0,20])
plt.ion()
plt.show()    
for i in range (3):
    dat.append(random.uniform(0,1))
    Ln.set_ydata(dat)
    Ln.set_xdata(range(len(dat)))
    plt.pause(1)

    print('done with loop')

done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
done with loop
